In [2]:
import sys
sys.path.append('../')
import numpy as np
from uihelper import PCLViewWidget, Draw3D, dc
%gui qt

## Create Plot Window and reference Drawing

In [3]:
plot_win = PCLViewWidget()
plot_win.show()
draw: Draw3D = plot_win.draw

The **Draw3D** is a class for iterative and sequential drawing, allowing the drawing of 3D points, 3D lines, and text.

```python
class Draw3D:

    def reset(self, update: bool = True) -> None: ...

    def update(self) -> None: ...

    def point(self,
        x: float,
        y: float,
        z: float,
        color: Tuple[float, float, float] = (0, 1, 0, 1),
        point_size: float = 0.05,
        update: bool = True,
    ) -> None: ....

    def points(
        self,
        points: np.ndarray,
        colors: Union[Tuple[float, float, float, float], np.ndarray] = (0.0, 1.0, 0.0, 1.0),
        points_size: float = 0.05,
        update: bool = True,
    ) -> None: ...

    def line(
        self,
        p1: Tuple[float, float, float],
        p2: Tuple[float, float, float],
        color: Tuple[float, float, float, float] = (0.0, 1.0, 0.0, 1.0),
        line_width: float = 1.0,
        update: bool = True,
    ) -> None: ....

    def lines(
        self,
        line_pts: np.ndarray,
        colors: Tuple[float, float, float, float] = (0.0, 1.0, 0.0, 1.0),
        line_width: float = 1.0,
        update: bool = True,
    ) -> None: ...

    def text(
        self,
        x: float,
        y: float,
        z: float,
        text: str,
        text_size: float = 10.0,
        text_color: Tuple[float, float, float, float] = (0.0, 1.0, 0.0, 1.0),
        update: bool = True,
    ) -> None: ...
```

## Reset (clear all drawings)

The reset is used to clear the scene and remove all previous drawings.

In [4]:
draw.reset()

## Draw Point Cloud

In [5]:
n = 1000
pts = np.random.rand(n, 3) * 5
pts_colors = np.random.rand(n, 4)
pts_colors[:, 3] = 0.75  # alpha color
plot_win.draw.reset(update=False) # 
plot_win.draw.points(pts, pts_colors, 0.2, update=True)

## Draw an Axis Lines

For drawing lines we need to provide the pair of vertices (3D points), and for the colors we need to provide a sequence of two RGBA colors in the range of [0, 1]

In [6]:
axis = np.array([[0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 1, 0], [0, 0, 0], [0, 0, 1]], dtype=np.float32)
axis_colors = np.array([[1, 0, 0, 1], [1, 0, 0, 1], [0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [0, 0, 1, 1],])

draw.reset(update=False)
draw.lines(axis, axis_colors) 

## Draw Text

We can text given the color.

In [7]:
axis = np.array([[0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 1, 0], [0, 0, 0], [0, 0, 1]], dtype=np.float32)
axis_colors = np.array([[1, 0, 0, 1], [1, 0, 0, 1], [0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [0, 0, 1, 1],])

draw.reset(update=False)
draw.lines(axis, axis_colors, update=False)

draw.text(0, 0, 0, "Origin", 20, (1, 1, 1, 1), update=False)

for point in axis[1::2]:
    draw.text(point[0], point[1], point[2], str(point), 20, (1, 1, 1, 1), update=False)
    
draw.update()  # update in the end


## Draw 3D Bounding Box

In [8]:
def create_bounding_box_edges(sx: float, sy: float, sz: float) -> np.ndarray:
    sx, sy, sz = sx / 2.0, sy / 2.0, sz / 2.0
    vertices = np.array([
        [-sx, -sy, -sz],
        [ sx, -sy, -sz],
        [-sx,  sy, -sz],
        [ sx,  sy, -sz],
        [-sx, -sy,  sz],
        [ sx, -sy,  sz],
        [-sx,  sy,  sz],
        [ sx,  sy,  sz]
    ])
    edges_indices = np.array([
        (0, 1), (1, 3), (3, 2), (2, 0),  # bottom face
        (4, 5), (5, 7), (7, 6), (6, 4),  # top face
        (0, 4), (1, 5), (2, 6), (3, 7)   # vertical edges
    ])
    return vertices[edges_indices].reshape(-1, 3)

bbox_edges1 = create_bounding_box_edges(1, 1, 1) + (0, 0, 0.5)
bbox_edges2 = create_bounding_box_edges(1, 1, 2) + (4, 0, 1)

draw.reset()
draw.lines(bbox_edges1, (1, 0, 0, 1))
draw.lines(bbox_edges2, (0, 1, 0, 1))

## Draw a Oriented Bounding Box 3D

In [9]:
def euler_to_rotation_matrix(roll, pitch, yaw):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])
    Ry = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])
    Rz = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    R = (Rz @ Ry) @ Rx
    return R


def transformation_matrix(t, r):
    M = np.eye(4, dtype=np.float32)
    M[:3,:3] = euler_to_rotation_matrix(r[0], r[1], r[2])
    M[:3,3] = t
    return M


def apply_transformation(points, T):
    homogeneous_points = np.hstack([points, np.ones((points.shape[0], 1))])
    transformed_homogeneous_points = homogeneous_points @ T.T
    transformed_points = transformed_homogeneous_points[:, :3].copy(order='C')
    return transformed_points


R1 = euler_to_rotation_matrix(np.radians(30), 0, np.radians(40))
bbox_edges1 = create_bounding_box_edges(2, 1.5, 1) + (0, 0, 0.5)
bbox_edges1 = bbox_edges1 @ R1.T + (0, 0, 0.5)

R2 = euler_to_rotation_matrix(np.radians(30), np.radians(20), np.radians(40))
bbox_edges2 = create_bounding_box_edges(2, 2, 2) + (1, 0, 1)
bbox_edges2 = bbox_edges2 @ R2.T + (0, 0, 0.5)

draw.reset()
draw.lines(bbox_edges1, (1, 0, 0, 1))
draw.lines(bbox_edges2, (0, 1, 0, 1))

## Single Track Model

In [10]:
def update_state(x, dt, u, L):
    """
    Update the state of the vehicle using the bicycle model.
    
    Parameters:
    x (np.array): Current state vector [x, y, theta, v].
    dt (float): Time step for integration.
    u (np.array): Control inputs [a, delta] where 'a' is acceleration and 'delta' is steering angle.
    L (float): Wheelbase of the vehicle.
    
    Returns:
    np.array: Updated state vector [x, y, theta, v].
    """
    # Current state
    x_pos, y_pos, theta, v = x
    
    # Control inputs
    a, delta = u
    
    # State derivatives
    x_dot = v * np.cos(theta)
    y_dot = v * np.sin(theta)
    theta_dot = v / L * np.tan(delta)
    v_dot = a
    
    # Update states using Euler integration
    x_pos_new = x_pos + x_dot * dt
    y_pos_new = y_pos + y_dot * dt
    theta_new = theta + theta_dot * dt
    v_new = v + v_dot * dt
    
    # New state vector
    return np.array([x_pos_new, y_pos_new, theta_new, v_new])

In [11]:
L = 2.5  # Wheelbase of the vehicle in meters
dt = 0.1  # Time step in seconds
initial_state = np.array([0.0, 0.0, 0.0, 10.0])  # Initial state [x, y, theta, v]
inputs = np.array([0.001, np.radians(5)])  # Inputs [acceleration, steering angle]

# Update state
new_state = update_state(initial_state, dt, inputs, L)

path_points = [(initial_state[0], initial_state[1], 0)]
path_points += [(new_state[0], new_state[1], 0)]

In [12]:
for t in range(0, 500, 1):
    x, y, yaw, v = new_state
    path_points.append([x, y, 0]) # type: ignore

    if t in [50, 100, 200, 250, 300, 410, 450, 490]:
        inputs[1] = -inputs[1]
    
    inputs[0] = 0
    draw.scale = 0.1

    new_state = update_state(new_state, dt, inputs, L)
    x, y, yaw, v = new_state
    path_points.append([x, y, 0]) # type: ignore

    if len(path_points) > 1000:
        n = len(path_points)
        path_points = path_points[n-1000:]

    R1 = euler_to_rotation_matrix(0, 0, yaw)
    bbox_edges1 = create_bounding_box_edges(L * 1.2, 1.6, 1.4)
    bbox_edges1 = bbox_edges1 @ R1.T + (x, y, 1.0)

    draw.reset(update=False)
    draw.lines(bbox_edges1, (0, 1, 0, 1), update=False)
    draw.lines(np.array(path_points), (1, 0, 0, 1))
    dc.process_events()

## Create a car with tyres

In [13]:
def unique_edges(edges):
    sorted_edges = np.sort(edges, axis=1)
    sorted_edges = sorted_edges[np.lexsort((sorted_edges[:, 1], sorted_edges[:, 0]))]
    unique_edges = np.unique(sorted_edges, axis=0)
    return unique_edges

def create_tyre(num_segments, height, radius):
    """
    Generate vertices and quad edges of a cylinder mesh.
    
    Parameters:
    - num_segments: int, number of segments in each circle.
    - height: float, height of the cylinder along the x-axis.
    - radius: float, radius of the cylinder along the y and z axes.
    
    Returns:
    - vertices: np.array, array of vertices.
    - quads: np.array, array of quad indices.
    """
    # Create angles for each segment
    angles = np.linspace(0, 2 * np.pi, num_segments, endpoint=False)
    
    # Compute x, y, z coordinates for vertices on the bottom circle
    bottom_circle = np.stack((radius * np.cos(angles),
                              np.zeros(num_segments), 
                              radius * np.sin(angles)), axis=-1)
    
    # Compute x, y, z coordinates for vertices on the top circle
    top_circle = np.stack((radius * np.cos(angles),
                           np.full(num_segments, height), 
                           radius * np.sin(angles)), axis=-1)
    
    # Combine vertices into a single array
    vertices = np.vstack((bottom_circle, top_circle))
    vertices[:, 1] -= height / 2.0
    
    # Create quad indices
    quads = []
    for i in range(num_segments):
        next_i = (i + 1) % num_segments
        # Quad vertices indices (bottom1, top1, top2, bottom2)
        bottom1 = i
        top1 = num_segments + i
        top2 = num_segments + next_i
        bottom2 = next_i
        quads += [(bottom1, top1)]
        quads += [(top1, top2)]
        quads += [(top2, bottom2)]
        quads += [(bottom2, bottom1)]
    
    quads = np.array(quads)
    quads = unique_edges(quads)
    return vertices, quads.reshape((-1,))



# create vehicle (two bbox)
L = 2.5
yaw = np.radians(20)
steering_angle = np.radians(10)
tire_rot_angle = np.radians(20)
pos = (0, 0, 0)

bbox_edges1 = create_bounding_box_edges(L * 1.6, 1.5, 0.7) + (0.05, 0, 0.5)
bbox_edges2 = create_bounding_box_edges(1.6, 1.4, 0.5) + (-0.3, 0, 1.10)
vehicle_chassis = np.vstack((bbox_edges1, bbox_edges2))

tire_vertices, tire_edges = create_tyre(16, 0.3, 0.35)
tire_lines = tire_vertices[tire_edges].reshape(-1, 3)

T_car = transformation_matrix(t=pos, r=(0,0,yaw))
T_fr = transformation_matrix(t=(L/2.0, -0.75, 0.35), r=(0, tire_rot_angle, steering_angle))
T_fl = transformation_matrix(t=(L/2.0, 0.75, 0.35), r=(0, tire_rot_angle, steering_angle))
T_rr = transformation_matrix(t=(-L/2.0, -0.75, 0.35), r=(0, tire_rot_angle, 0))
T_rl = transformation_matrix(t=(-L/2.0, 0.75, 0.35), r=(0, tire_rot_angle, 0))

Car = apply_transformation(vehicle_chassis, T_car)
Tire1 = apply_transformation(tire_lines, T_car @ T_fl)
Tire2 = apply_transformation(tire_lines, T_car @ T_fr)
Tire3 = apply_transformation(tire_lines, T_car @ T_rl)
Tire4 = apply_transformation(tire_lines, T_car @ T_rr)

draw.reset()
draw.lines(Car, (0, 1, 0, 0.8))
draw.lines(Tire1, (0, 0.5, 0.5, 1.0))
draw.lines(Tire2, (0, 0.5, 0.5, 1.0))
draw.lines(Tire3, (0, 0.5, 0.5, 1.0))
draw.lines(Tire4, (0, 0.5, 0.5, 1.0))
draw.lines(axis, axis_colors) 

In [14]:
## apply dynamics

L = 2.5  # Wheelbase of the vehicle in meters
dt = 0.01  # Time step in seconds
initial_state = np.array([0.0, 0.0, 0.0, 10.0])  # Initial state [x, y, theta, v]
inputs = np.array([0.01, np.radians(10)])  # Inputs [acceleration, steering angle]

# Update state
new_state = update_state(initial_state, dt, inputs, L)

path_points = [(initial_state[0], initial_state[1], 0)]
path_points += [(new_state[0], new_state[1], 0)]
inputs[0] = 0

In [15]:
tire_rot_angle = 0
for t in range(0, 400, 1):
    tire_rot_angle += 0.2
    x, y, yaw, v = new_state
    path_points.append([x, y, 0]) # type: ignore
    
    
    if t < 100:
        inputs[1] += 0.01
        inputs[1] = min(inputs[1], 0.2)
    elif t < 200:
        inputs[1] -= 0.01
        inputs[1] = max(inputs[1], -0.2)
    elif t < 300:
        inputs[1] += 0.02
        inputs[1] = min(inputs[1], -0.2)
    else:
        inputs[1] -= 0.02
        inputs[1] = max(inputs[1], -0.3)
    
    draw.scale = 0.1

    steering_angle = inputs[1]
    
    new_state = update_state(new_state, dt, inputs, L)
    x, y, yaw, v = new_state
    path_points.append([x, y, 0]) # type: ignore

    if len(path_points) > 1000:
        n = len(path_points)
        path_points = path_points[n-1000:]

    T_car = transformation_matrix(t=(x, y, 0), r=(0,0, yaw))
    T_fr = transformation_matrix(t=(L/2.0, -0.75, 0.35), r=(0, tire_rot_angle, steering_angle))
    T_fl = transformation_matrix(t=(L/2.0, 0.75, 0.35), r=(0, tire_rot_angle, steering_angle))
    T_rr = transformation_matrix(t=(-L/2.0, -0.75, 0.35), r=(0, tire_rot_angle, 0))
    T_rl = transformation_matrix(t=(-L/2.0, 0.75, 0.35), r=(0, tire_rot_angle, 0))

    Car = apply_transformation(vehicle_chassis, T_car)
    Tire1 = apply_transformation(tire_lines, T_car @ T_fl)
    Tire2 = apply_transformation(tire_lines, T_car @ T_fr)
    Tire3 = apply_transformation(tire_lines, T_car @ T_rl)
    Tire4 = apply_transformation(tire_lines, T_car @ T_rr)

    draw.reset()
    draw.lines(Car, (0, 1, 0, 0.8))
    draw.lines(Tire1, (0, 0.5, 0.5, 1.0))
    draw.lines(Tire2, (0, 0.5, 0.5, 1.0))
    draw.lines(Tire3, (0, 0.5, 0.5, 1.0))
    draw.lines(Tire4, (0, 0.5, 0.5, 1.0))
    draw.lines(axis, axis_colors) 
    
    draw.lines(np.array(path_points), (1, 0, 0, 1))
    
    dc.process_events()